## Imports

In [462]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import talib
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler

## Functions

In [463]:
def load_data(data_path):
    df = pd.read_csv(data_path)
    return df


def preprocess_data(df):
    df["color_change"] = df["color"].diff().ne(0).astype(int)
    df["color_change"].fillna(0, inplace=True)

    return df


def scale_data(df):
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df_scaled


def add_heiken_ashi_features(df):
    # Create Heiken Ashi DataFrame
    ha_df = df.ta.ha()

    # Rename the HA columns
    ha_df.columns = [f"HA_{col}" for col in ha_df.columns]

    # Join the HA columns to the original dataframe
    df = df.join(ha_df)

    # Heiken Ashi Close to Open
    df["HA_close_open"] = df["HA_close"] - df["HA_open"]

    # Heiken Ashi High Low Range
    df["HA_high_low"] = df["HA_high"] - df["HA_low"]

    # Heiken Ashi Body Range
    df["HA_body"] = abs(df["HA_close"] - df["HA_open"])

    # Heiken Ashi Price Direction
    df["HA_direction"] = (df["HA_close"] > df["HA_open"]).astype(int)

    # Heiken Ashi Volume-weighted Price
    df["HA_vwap"] = (df["HA_close"] * df["volume"]).cumsum() / df["volume"].cumsum()

    # Lag 1 feature
    df["HA_close_lag1"] = df["HA_close"].shift(1)

    # Close Change
    df["HA_close_change"] = df["HA_close"].diff()

    # Close % Change
    df["HA_close_pct_change"] = df["HA_close"].pct_change()

    # 5-period Simple Moving Average
    df["HA_sma5"] = df["HA_close"].rolling(5).mean()

    # 5-period Exponential Moving Average
    df["HA_ema5"] = df["HA_close"].ewm(span=5).mean()

    # Additional features
    df["HA_ema10"] = df["HA_close"].ewm(span=10).mean()
    df["HA_ema15"] = df["HA_close"].ewm(span=15).mean()
    df["HA_pct_diff_ema5_15"] = (
        (df["HA_ema5"] - df["HA_ema15"]) / df["HA_ema15"]
    ) * 100
    df["HA_rsi"] = ta.rsi(df["HA_close"])
    # Calculate Short Term Exponential Moving Average
    df["short_ema"] = df["HA_close"].ewm(span=12, adjust=False).mean()

    # Calculate Long Term Exponential Moving Average
    df["long_ema"] = df["HA_close"].ewm(span=26, adjust=False).mean()

    # Calculate Moving Average Convergence Divergence (MACD)
    df["HA_macd"] = df["short_ema"] - df["long_ema"]

    # Calculate Signal Line
    df["HA_macds"] = df["HA_macd"].ewm(span=9, adjust=False).mean()

    # Calculate MACD Histogram
    df["HA_macdh"] = df["HA_macd"] - df["HA_macds"]

    # Drop temporary short_ema and long_ema columns
    df = df.drop(["short_ema", "long_ema"], axis=1)

    df["HA_cci"] = ta.cci(df["HA_high"], df["HA_low"], df["HA_close"])
    df["HA_atr"] = ta.atr(df["HA_high"], df["HA_low"], df["HA_close"])
    df["HA_ha_close_bbp50_std"] = (
        ta.stdev(df["HA_close"], 50) / df["HA_close"]
    )  # Bollinger Bands normalized width
    df["HA_mfi"] = ta.mfi(df["HA_high"], df["HA_low"], df["HA_close"], df["volume"])

    return df


def compute_renko(df, brick_size):
    renko_df = pd.DataFrame(index=df.index, columns=["open", "high", "low", "close"])
    current_price = df["close"][0]
    last_reset_price = current_price

    for idx in df.index[1:]:
        current_price = df.loc[idx, "close"]

        while last_reset_price + brick_size <= current_price:
            renko_df.loc[idx] = [
                last_reset_price,
                last_reset_price + brick_size,
                last_reset_price,
                last_reset_price + brick_size,
            ]
            last_reset_price += brick_size

        while last_reset_price - brick_size >= current_price:
            renko_df.loc[idx] = [
                last_reset_price,
                last_reset_price - brick_size,
                last_reset_price - brick_size,
                last_reset_price,
            ]
            last_reset_price -= brick_size

    return renko_df.dropna()


def add_renko_features(df, renko_df):
    # Append "_renko" to the column names of renko_df
    renko_df.columns = [str(col) + "_renko" for col in renko_df.columns]

    # Add EMA, SMA, WMA, RSI, and Bollinger Bands to renko_df
    renko_df["sma_renko"] = talib.SMA(renko_df["close_renko"], timeperiod=5)
    renko_df["ema_renko"] = talib.EMA(renko_df["close_renko"], timeperiod=5)
    renko_df["wma_renko"] = talib.WMA(renko_df["close_renko"], timeperiod=5)
    renko_df["rsi_renko"] = talib.RSI(renko_df["close_renko"], timeperiod=5)
    (
        renko_df["upper_band_renko"],
        renko_df["middle_band_renko"],
        renko_df["lower_band_renko"],
    ) = talib.BBANDS(
        renko_df["close_renko"], timeperiod=5
    )  # Join Renko data with the original DataFrame
    df = df.join(renko_df)

    # Derived features
    df["close_open_renko"] = df["close_renko"] - df["open_renko"]
    df["high_low_renko"] = df["high_renko"] - df["low_renko"]
    df["close_renko_lag1"] = df["close_renko"].shift(1)
    df["close_change_renko"] = df["close_renko"].diff()
    df["renko_open_lag1"] = df["open_renko"].shift(1)
    df["renko_high_lag1"] = df["high_renko"].shift(1)
    df["renko_low_lag1"] = df["low_renko"].shift(1)
    df["renko_volume_lag1"] = df["volume"].shift(1)
    # Compute the 'renko_uptrend' feature
    df["renko_uptrend"] = df["close_renko"].diff().apply(lambda x: int(x > 0))

    # Fill NaN values using forward fill and backward fill
    df["renko_uptrend"].ffill(inplace=True)
    df["renko_uptrend"].bfill(inplace=True)
    df["direction_renko"] = df["renko_uptrend"].astype(int)
    df["direction_change"] = df["direction_renko"].diff().abs()

    return df


def kagi(df):
    timestamps = [df.index[0]]  # Store timestamps for the kagi_df index
    kagi_data = []
    cur_col = {"dir": "up", "start": df["close"].iloc[0]}

    for index, row in df.iterrows():
        cp = row["close"]
        diff = cp - cur_col["start"]

        if cur_col["dir"] == "up":
            if diff < 0:
                kagi_data.append(cur_col)
                cur_col = {"dir": "down", "start": cp}
                timestamps.append(index)
            else:
                cur_col["start"] += diff
        else:
            if diff > 0:
                kagi_data.append(cur_col)
                cur_col = {"dir": "up", "start": cp}
                timestamps.append(index)
            else:
                cur_col["start"] -= diff

    kagi_data.append(cur_col)
    kagi_df = pd.DataFrame(kagi_data, index=timestamps)
    return kagi_df


def add_kagi_features(df, kagi_df):
    kagi_df["sma"] = talib.SMA(kagi_df["start"], 5)
    kagi_df["ema"] = talib.EMA(kagi_df["start"], 5)
    kagi_df["wma"] = talib.WMA(kagi_df["start"], 5)
    kagi_df["macd"], kagi_df["macdsig"], kagi_df["macdhist"] = talib.MACD(
        kagi_df["start"], 12, 26, 9
    )
    kagi_df["rsi"] = talib.RSI(kagi_df["start"], 14)
    kagi_df["upper"], kagi_df["mid"], kagi_df["lower"] = talib.BBANDS(
        kagi_df["start"], 5
    )
    kagi_df["adx"] = talib.ADX(kagi_df["start"], kagi_df["start"], kagi_df["start"], 14)
    kagi_df["atr"] = talib.ATR(kagi_df["start"], kagi_df["start"], kagi_df["start"], 14)
    kagi_df["slowk"], kagi_df["slowd"] = talib.STOCH(
        kagi_df["start"], kagi_df["start"], kagi_df["start"], 5, 3, 0, 3, 0
    )
    # Align the volume series with kagi_df
    aligned_volume = df["volume"].reindex(kagi_df.index, method="pad")

    # Calculate Accumulation / Distribution Line
    kagi_df["ad"] = talib.AD(
        kagi_df["start"], kagi_df["start"], kagi_df["start"], aligned_volume
    )

    # Calculate On Balance Volume
    obv = talib.OBV(kagi_df["start"], aligned_volume)
    kagi_df["obv"] = (obv - obv.min()) / (obv.max() - obv.min())

    kagi_df["roc"] = talib.ROC(kagi_df["start"], 10)
    return kagi_df.add_prefix("kagi_")


def point_and_figure(df, box_size=1, reversal=3):
    pnf_data = []
    current_column = {"direction": "up", "start_price": df["close"].iloc[0], "boxes": 1}

    for index, row in df.iterrows():
        current_price = row["close"]
        box_diff = abs(current_price - current_column["start_price"]) / box_size

        if current_column["direction"] == "up":
            if box_diff >= reversal and current_price < current_column["start_price"]:
                pnf_data.append(current_column)
                current_column = {
                    "direction": "down",
                    "start_price": current_price,
                    "boxes": 1,
                }
            elif box_diff >= 1:
                current_column["boxes"] += int(box_diff)
                current_column["start_price"] += int(box_diff) * box_size
        else:
            if box_diff >= reversal and current_price > current_column["start_price"]:
                pnf_data.append(current_column)
                current_column = {
                    "direction": "up",
                    "start_price": current_price,
                    "boxes": 1,
                }
            elif box_diff >= 1:
                current_column["boxes"] += int(box_diff)
                current_column["start_price"] -= int(box_diff) * box_size

    pnf_data.append(current_column)
    pnf_df = pd.DataFrame(pnf_data)
    pnf_df["time"] = df.index
    return pnf_df


def add_point_and_figure_features(pnf_df):
    # Calculate moving averages
    pnf_df["sma"] = talib.SMA(pnf_df["start_price"], timeperiod=5)
    pnf_df["ema"] = talib.EMA(pnf_df["start_price"], timeperiod=5)
    pnf_df["wma"] = talib.WMA(pnf_df["start_price"], timeperiod=5)

    # Calculate MACD
    pnf_df["macd"], pnf_df["macdsignal"], pnf_df["macdhist"] = talib.MACD(
        pnf_df["start_price"], fastperiod=12, slowperiod=26, signalperiod=9
    )

    # Calculate RSI
    pnf_df["rsi"] = talib.RSI(pnf_df["start_price"], timeperiod=14)

    # Calculate Bollinger Bands
    pnf_df["upper_band"], pnf_df["middle_band"], pnf_df["lower_band"] = talib.BBANDS(
        pnf_df["start_price"], timeperiod=5
    )

    # Calculate ADX
    pnf_df["adx"] = talib.ADX(
        pnf_df["start_price"],
        pnf_df["start_price"],
        pnf_df["start_price"],
        timeperiod=14,
    )

    # Calculate ATR
    pnf_df["atr"] = talib.ATR(
        pnf_df["start_price"],
        pnf_df["start_price"],
        pnf_df["start_price"],
        timeperiod=14,
    )

    # Calculate Stochastic Oscillator
    pnf_df["slowk"], pnf_df["slowd"] = talib.STOCH(
        pnf_df["start_price"],
        pnf_df["start_price"],
        pnf_df["start_price"],
        fastk_period=5,
        slowk_period=3,
        slowk_matype=0,
        slowd_period=3,
        slowd_matype=0,
    )

    # Calculate Accumulation / Distribution Line
    pnf_df["ad"] = talib.AD(
        pnf_df["start_price"],
        pnf_df["start_price"],
        pnf_df["start_price"],
        pnf_df["boxes"],
    )

    # Calculate On Balance Volume
    obv = talib.OBV(pnf_df["start_price"], pnf_df["boxes"])
    pnf_df["obv"] = (obv - obv.min()) / (obv.max() - obv.min())

    # Calculate Rate of Change
    pnf_df["roc"] = talib.ROC(pnf_df["start_price"], timeperiod=10)

    return pnf_df.add_prefix("pnf_")


# def three_line_break(df, reversal=3):
#     tlb_data = [{"direction": "up", "start_price": df.iloc[0, :], "boxes": 1}]
#     for index, row in df.iterrows():
#         last_line = tlb_data[-1]

#         if last_line["direction"] == "up":
#             if row["low"] < last_line["start_price"]["low"]:
#                 last_line["boxes"] += 1
#             elif row["high"] >= last_line["start_price"]["high"] * (1 + reversal / 100):
#                 tlb_data.append({"direction": "down", "start_price": row, "boxes": 1})
#         else:
#             if row["high"] > last_line["start_price"]["high"]:
#                 last_line["boxes"] += 1
#             elif row["low"] <= last_line["start_price"]["low"] * (1 - reversal / 100):
#                 tlb_data.append({"direction": "up", "start_price": row, "boxes": 1})
#     return pd.DataFrame(tlb_data)


# def add_three_line_break_features(tlb_df):
#     close_prices = tlb_df["start_price"].apply(lambda x: x["close"])
#     high_prices = tlb_df["start_price"].apply(lambda x: x["high"])
#     low_prices = tlb_df["start_price"].apply(lambda x: x["low"])

#     # Calculate moving averages
#     tlb_df["sma"] = talib.SMA(close_prices, timeperiod=5)
#     tlb_df["ema"] = talib.EMA(close_prices, timeperiod=5)
#     tlb_df["wma"] = talib.WMA(close_prices, timeperiod=5)

#     # Calculate MACD
#     tlb_df["macd"], tlb_df["macdsignal"], tlb_df["macdhist"] = talib.MACD(
#         close_prices, fastperiod=12, slowperiod=26, signalperiod=9
#     )

#     # Calculate RSI
#     tlb_df["rsi"] = talib.RSI(close_prices, timeperiod=14)

#     # Calculate Bollinger Bands
#     tlb_df["upper_band"], tlb_df["middle_band"], tlb_df["lower_band"] = talib.BBANDS(
#         close_prices, timeperiod=5
#     )

#     # Calculate ADX
#     tlb_df["adx"] = talib.ADX(high_prices, low_prices, close_prices, timeperiod=14)

#     # Calculate ATR
#     tlb_df["atr"] = talib.ATR(high_prices, low_prices, close_prices, timeperiod=14)

#     # Calculate Stochastic Oscillator
#     tlb_df["slowk"], tlb_df["slowd"] = talib.STOCH(
#         high_prices,
#         low_prices,
#         close_prices,
#         fastk_period=5,
#         slowk_period=3,
#         slowk_matype=0,
#         slowd_period=3,
#         slowd_matype=0,
#     )

#     # Calculate Accumulation / Distribution Line
#     volume = tlb_df["start_price"].apply(lambda x: x["volume"])
#     tlb_df["ad"] = talib.AD(high_prices, low_prices, close_prices, volume)

#     # Calculate On Balance Volume
#     tlb_df["obv"] = talib.OBV(close_prices, volume)

#     # Calculate Rate of Change
#     tlb_df["roc"] = talib.ROC(close_prices, timeperiod=10)

#     return tlb_df.add_prefix("tlb_")


def tlb(df, r=3):
    data = [{"d": "up", "sp": df.iloc[0, :], "bx": 1}]
    for i, row in df.iterrows():
        ll = data[-1]
        if ll["d"] == "up":
            if row["low"] < ll["sp"]["low"]:
                ll["bx"] += 1
            elif row["high"] >= ll["sp"]["high"] * (1 + r / 100):
                data.append({"d": "down", "sp": row, "bx": 1})
        else:
            if row["high"] > ll["sp"]["high"]:
                ll["bx"] += 1
            elif row["low"] <= ll["sp"]["low"] * (1 - r / 100):
                data.append({"d": "up", "sp": row, "bx": 1})
    return pd.DataFrame(data)


def add_tlb_ft(tlb_df):
    p = lambda x: (x["close"], x["high"], x["low"], x["volume"])
    cl_hl_lv = tlb_df["sp"].apply(p)
    c, h, l, v = map(np.array, zip(*cl_hl_lv))

    ft = {}
    for fn, tp in (
        ("SMA", 5),
        ("EMA", 5),
        ("WMA", 5),
        ("MACD", None),
        ("RSI", 14),
        ("upper_band", 5),
        ("ADX", 14),
        ("ATR", 14),
        ("slowk", None),
        ("AD", None),
        ("OBV", None),
        ("ROC", 10),
    ):
        if fn in ["SMA", "EMA", "WMA"]:
            ft[fn] = getattr(talib, fn)(c, tp)
        elif fn == "MACD":
            ft["macd"], ft["macdsignal"], ft["macdhist"] = talib.MACD(c, 12, 26, 9)
        elif fn == "upper_band":
            ft[fn], ft["middle_band"], ft["lower_band"] = talib.BBANDS(c, tp)
        elif fn == "slowk":
            ft["slowk"], ft["slowd"] = talib.STOCH(h, l, c, 5, 3, 0, 3, 0)
        elif fn == "AD":
            ft[fn] = talib.AD(h, l, c, v)
        elif fn == "OBV":
            ft[fn] = talib.OBV(c, v)
        else:
            ft[fn] = getattr(talib, fn)(c, tp)
    ft = pd.DataFrame(ft)
    return ft.add_prefix("tlb_")


def create_features(df):
    ## DIFFERENCES ##

    # Price differences
    df["price_diff"] = df["close"].diff()
    df["op_cl_diff"] = df["open"] - df["close"]

    # Moving averages
    df["ma_5"] = df["close"].rolling(window=5).mean()
    df["ma_10"] = df["close"].rolling(window=10).mean()

    # Price percentage change
    df["pct_change"] = df["close"].pct_change()

    # RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df["rsi"] = 100 - 100 / (1 + rs)

    # Other popular difference features
    for i in range(1, 35):
        df["diff_{}".format(i)] = df["close"].diff(i)

    sma = df["close"].rolling(window=20).mean()
    std = df["close"].rolling(window=20).std()
    df["upper_band"] = sma + (2 * std)
    df["lower_band"] = sma - (2 * std)

    highest_high = df["high"].rolling(window=14).max()
    lowest_low = df["low"].rolling(window=14).min()
    df["williams_r"] = (highest_high - df["close"]) / (highest_high - lowest_low) * -100

    df["obv"] = (np.sign(df["close"].diff()) * df["volume"]).fillna(0).cumsum()

    tp = (df["high"] + df["low"] + df["close"]) / 3
    sma = tp.rolling(window=20).mean()
    mean_deviation = tp.rolling(window=20).apply(
        lambda x: np.mean(np.abs(x - x.mean()))
    )
    df["cci"] = (tp - sma) / (0.015 * mean_deviation)

    true_range = pd.concat(
        [
            df["high"] - df["low"],
            (df["high"] - df["close"].shift()).abs(),
            (df["close"].shift() - df["low"]).abs(),
        ],
        axis=1,
    ).max(axis=1)
    df["atr"] = true_range.rolling(window=14).mean()

    money_flow_vol = (
        ((df["close"] - df["low"]) - (df["high"] - df["close"]))
        / (df["high"] - df["low"])
        * df["volume"]
    )
    cmf = (
        money_flow_vol.rolling(window=20).sum() / df["volume"].rolling(window=20).sum()
    )
    df["cmf"] = cmf

    ema_12 = df["close"].ewm(span=12).mean()
    ema_26 = df["close"].ewm(span=26).mean()
    ppo = (ema_12 - ema_26) / ema_26 * 100
    df["ppo"] = ppo

    ### LAG ###

    # Lag closing prices
    for i in range(1, 11):
        df["lag_close_{}".format(i)] = df["close"].shift(i)

    # Lag daily returns
    returns = df["close"].pct_change()
    for i in range(1, 6):
        df["lag_return_{}".format(i)] = returns.shift(i)

    # Lag high and low prices
    for i in range(1, 4):
        df["lag_high_{}".format(i)] = df["high"].shift(i)
        df["lag_low_{}".format(i)] = df["low"].shift(i)

    # Historical volatility
    df["hist_volatility_10"] = returns.rolling(window=10).std()
    df["hist_volatility_20"] = returns.rolling(window=20).std()
    df["hist_volatility_30"] = returns.rolling(window=30).std()

    # Previous day's RSI value
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - 100 / (1 + rs)
    df["lag_rsi_1"] = rsi.shift(1)

    ### ROLLING ###

    # Moving averages
    df["ma_5"] = df["close"].rolling(window=5).mean()
    df["ma_10"] = df["close"].rolling(window=10).mean()
    df["ma_20"] = df["close"].rolling(window=20).mean()

    # Exponential moving averages
    df["ema_5"] = df["close"].ewm(span=5).mean()
    df["ema_10"] = df["close"].ewm(span=10).mean()
    df["ema_20"] = df["close"].ewm(span=20).mean()

    # Bollinger Bands
    sma = df["close"].rolling(window=20).mean()
    std = df["close"].rolling(window=20).std()
    df["upper_band"] = sma + (2 * std)
    df["lower_band"] = sma - (2 * std)

    # Rate of Change (ROC)
    df["roc_5"] = df["close"].pct_change(periods=5)
    df["roc_10"] = df["close"].pct_change(periods=10)

    # Standard deviation
    df["std_5"] = df["close"].rolling(window=5).std()
    df["std_10"] = df["close"].rolling(window=10).std()

    # Average True Range (ATR)
    true_range = pd.concat(
        [
            df["high"] - df["low"],
            (df["high"] - df["close"].shift()).abs(),
            (df["low"] - df["close"].shift()).abs(),
        ],
        axis=1,
    ).max(axis=1)
    df["atr_14"] = true_range.rolling(window=14).mean()

    # Keltner Channels
    middle_line = df["close"].rolling(window=20).mean()
    upper_keltner = middle_line + 2 * df["atr_14"]
    lower_keltner = middle_line - 2 * df["atr_14"]
    df["upper_keltner"] = upper_keltner
    df["lower_keltner"] = lower_keltner

    # RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df["rsi"] = 100 - 100 / (1 + rs)

    ### RATIO ###

    # Calculate some basic features first
    returns = df["close"].pct_change()
    ema_12 = df["close"].ewm(span=12).mean()
    ema_26 = df["close"].ewm(span=26).mean()

    # 1. Price-to-moving-average ratios
    df["price_to_ma_5"] = df["close"] / df["ma_5"]
    df["price_to_ma_10"] = df["close"] / df["ma_10"]
    df["price_to_ma_20"] = df["close"] / df["ma_20"]

    # 2. Price-to-EMA ratios
    df["price_to_ema_5"] = df["close"] / df["ema_5"]
    df["price_to_ema_10"] = df["close"] / df["ema_10"]
    df["price_to_ema_20"] = df["close"] / df["ema_20"]

    # 3. EMA-to-moving-average ratios
    df["ema_to_ma_5"] = df["ema_5"] / df["ma_5"]
    df["ema_to_ma_10"] = df["ema_10"] / df["ma_10"]
    df["ema_to_ma_20"] = df["ema_20"] / df["ma_20"]

    # 4. EMA-to-EMA ratios
    df["ema5_to_ema10"] = df["ema_5"] / df["ema_10"]
    df["ema5_to_ema20"] = df["ema_5"] / df["ema_20"]
    df["ema10_to_ema20"] = df["ema_10"] / df["ema_20"]

    # 5. Moving-average-to-moving-average ratios
    df["ma5_to_ma10"] = df["ma_5"] / df["ma_10"]
    df["ma5_to_ma20"] = df["ma_5"] / df["ma_20"]
    df["ma10_to_ma20"] = df["ma_10"] / df["ma_20"]

    # 6. Price-to-Bollinger-Band ratios
    df["price_to_upper_band"] = df["close"] / df["upper_band"]
    df["price_to_lower_band"] = df["close"] / df["lower_band"]

    # 7. Price-to-Keltner-Channel ratios
    df["price_to_upper_keltner"] = df["close"] / df["upper_keltner"]
    df["price_to_lower_keltner"] = df["close"] / df["lower_keltner"]

    # 8. Price-to-previous-day-high ratio
    df["price_to_prev_high"] = df["close"] / df["high"].shift(1)

    # 9. Price-to-previous-day-low ratio
    df["price_to_prev_low"] = df["close"] / df["low"].shift(1)

    # 10. RSI-to-moving-average ratio
    df["rsi_to_ma_5"] = df["rsi"] / df["ma_5"]

    ### CORRELATION ####

    # Calculate some basic features first
    returns = df["close"].pct_change()
    volume_change = df["volume"].pct_change()

    # 1. Correlation with returns and volume (short-term)
    df["return_volume_corr_5"] = returns.rolling(window=5).corr(df["volume"])
    df["return_volume_corr_10"] = returns.rolling(window=10).corr(df["volume"])
    df["return_volume_corr_20"] = returns.rolling(window=20).corr(df["volume"])

    # 2. Correlation with returns and volume (medium-term)
    df["return_volume_corr_50"] = returns.rolling(window=50).corr(df["volume"])
    df["return_volume_corr_100"] = returns.rolling(window=100).corr(df["volume"])

    # 3. Correlation with price and volume
    df["price_volume_corr_5"] = df["close"].rolling(window=5).corr(df["volume"])
    df["price_volume_corr_10"] = df["close"].rolling(window=10).corr(df["volume"])
    df["price_volume_corr_20"] = df["close"].rolling(window=20).corr(df["volume"])

    # 4. Correlation with price and volume (medium-term)
    df["price_volume_corr_50"] = df["close"].rolling(window=50).corr(df["volume"])
    df["price_volume_corr_100"] = df["close"].rolling(window=100).corr(df["volume"])

    # 5. Price / volume
    df["price_to_volume"] = df["close"] / df["volume"]

    # 6. Volume Relative Strength Index (VRSI)
    delta_vol = df["volume"].diff()
    gain_vol = delta_vol.where(delta_vol > 0, 0)
    loss_vol = -delta_vol.where(delta_vol < 0, 0)
    avg_gain_vol = gain_vol.rolling(window=14).mean()
    avg_loss_vol = loss_vol.rolling(window=14).mean()
    rs_vol = avg_gain_vol / avg_loss_vol
    df["vrsi"] = 100 - 100 / (1 + rs_vol)

    # 7. Change in volume
    df["volume_change"] = volume_change

    # 8. Moving averages of volume
    df["volume_ma_5"] = df["volume"].rolling(window=5).mean()
    df["volume_ma_10"] = df["volume"].rolling(window=10).mean()
    df["volume_ma_20"] = df["volume"].rolling(window=20).mean()

    # 9. Standard deviation of volume
    df["volume_std_5"] = df["volume"].rolling(window=5).std()
    df["volume_std_10"] = df["volume"].rolling(window=10).std()
    df["volume_std_20"] = df["volume"].rolling(window=20).std()

    # 10. Ratio of volume to moving average volume
    df["volume_to_ma_volume"] = df["volume"] / df["volume_ma_20"]

    ### EXTRA ###

    # Calculate necessary base features
    returns = df["close"].pct_change()
    delta = df["close"].diff()

    # close_change
    df["close_change"] = df["close"].diff()

    # high_pct
    df["high_pct"] = df["high"].pct_change()

    # close_change_roll5
    df["close_change_roll5"] = df["close_change"].rolling(window=5).mean()

    # RSI_14_roll5
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - 100 / (1 + rs)
    df["RSI_14_roll5"] = rsi.rolling(window=5).mean()

    # ATR_14_roll5
    true_range = pd.concat(
        [
            df["high"] - df["low"],
            (df["high"] - df["close"].shift()).abs(),
            (df["low"] - df["close"].shift()).abs(),
        ],
        axis=1,
    ).max(axis=1)
    df["ATR_14_roll5"] = true_range.rolling(window=14).mean().rolling(window=5).mean()

    # volume_roll5
    df["volume_roll5"] = df["volume"].rolling(window=5).mean()

    # high_pct_roll5
    df["high_pct_roll5"] = df["high_pct"].rolling(window=5).mean()

    # volatility_5
    df["volatility_5"] = df["close"].rolling(window=5).std()

    # price_ema5
    df["price_ema5"] = df["close"].ewm(span=5).mean()

    # volume_ema5
    df["volume_ema5"] = df["volume"].ewm(span=5).mean()

    # price_to_ema5
    df["price_to_ema5"] = df["close"] / df["price_ema5"]

    # volume_change_roll5
    df["volume_change_roll5"] = volume_change.rolling(window=5).mean()

    # avg_vol_last_100
    df["avg_vol_last_100"] = df["volume"].rolling(window=100).mean()

    # turnover
    df["turnover"] = df["volume"] * df["close"]

    return df


def timeseries_cv_score(X, y, n_splits):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    f1_scores = []
    auc_scores = []  # list to store ROC AUC scores for each split
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Define LSTM model
        model = Sequential()
        model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dense(1, activation="sigmoid"))  # because of binary classification

        model.compile(
            loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
        )

        # Train the model
        model.fit(X_train, y_train, epochs=10, verbose=0)

        # Make predictions on the test set
        y_pred = model.predict(X_test).ravel()

        # Calculate F1 score of the model on the test set
        f1 = f1_score(y_test, (y_pred > 0.5).astype("int32"))
        f1_scores.append(f1)

        # Calculate ROC AUC score of the model on the test set
        auc = roc_auc_score(y_test, y_pred)
        auc_scores.append(auc)

    return np.mean(f1_scores), np.mean(auc_scores)

## Globals

In [464]:
# Set display options to show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# Prepare TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)
# Load the data
data_path = "../../../data/kc/btc/raw/kc_btc_15min.csv"

## Preprocessing

In [465]:
# Load and preprocess the data
df = load_data(data_path)
df = preprocess_data(df)

print(df.columns)

# Timestamp conversion and index setting
df["time_unix"] = df["time"]
df["time"] = pd.to_datetime(df["time"], unit="s")  # Assuming 'time' is in seconds
df.set_index("time", inplace=True)
df = df.loc[~df.index.duplicated(keep="first")]

# Technical Analysis features
# df.ta.strategy("all")

# Check your results and exclude as necessary.
df.ta.strategy(fast=10, slow=50, verbose=True)

# Chart Transformation features
df = add_heiken_ashi_features(df)
print(df.columns)

brick_size = 1
renko_df = compute_renko(df, brick_size)
df = add_renko_features(df, renko_df)
df = df.merge(renko_df, left_on="time", right_on="time")
kagi_df = kagi(df)
kagi_df = add_kagi_features(df, kagi_df)
df = df.merge(kagi_df, left_index=True, right_index=True)
tlb_df = tlb(df)
tlb_df = add_tlb_ft(tlb_df)
df = df.merge(tlb_df, left_index=True, right_index=True)
# pnf_df = point_and_figure(df, box_size=1, reversal=3)
# pnf_df = add_point_and_figure_features(pnf_df)
# df = df.merge(pnf_df, on="time", how="outer")
# tlb_df = three_line_break(df, reversal=3)
# tlb_df = add_three_line_break_features(tlb_df)

df.tail()

# # Additional features
# df = create_features(df)

# # Now extract additional date information from the original dataframe
# df["minute"] = df.index.minute
# df["hour"] = df.index.hour
# df["day"] = df.index.day
# df["month"] = df.index.month

# # Sanity check. Make sure all the columns and types
# # print(df.columns)

# # Forward Fill
# df.ffill(inplace=True)

# # Backward Fill
# df.bfill(inplace=True)

# # # Finding problem features for standard scalar
# # non_num_features = df.select_dtypes(exclude=["int32", "int64", "float32", "float64"])
# # print("Fix these features:\n")
# # for col, dtype in non_num_features.dtypes.items():
# #     print(f"{col}: {dtype}")


# # Select numeric columns which need to be scaled
# do_not_scale_columns = [
#     "time_unix",
#     "minute",
#     "hour",
#     "day",
#     "month",
# ]
# scaler = StandardScaler()
# for col in df.columns:
#     if col not in do_not_scale_columns:
#         df[col] = scaler.fit_transform(df[[col]])


# X = df.drop("color_change", axis=1)
# y = df["color_change"]

# df.tail()
# duplicate features
# duplicated_features = df.columns.duplicated()
# print("Duplicate Features: ", df.columns[duplicated_features])
# Total features
# print("Total features in DataFrame: ", df.shape[1])

Index(['open', 'close', 'high', 'low', 'volume', 'color', 'time',
       'color_change'],
      dtype='object')
[+] Strategy: All
[i] Indicator arguments: {'fast': 10, 'slow': 50, 'append': True}
[i] Excluded[12]: above, above_value, below, below_value, cross, cross_value, long_run, short_run, td_seq, tsignals, vp, xsignals
[i] Multiprocessing 131 indicators with 3 chunks and 12/12 cpus.


131it [00:02, 51.80it/s]


[i] Total indicators: 131
[i] Columns added: 278
[i] Last Run: Tuesday June 20, 2023, NYSE: 19:12:40, Local: 23:12:40 Pacific Daylight Time, Day 171/365 (47.00%)
Index(['open', 'close', 'high', 'low', 'volume', 'color', 'color_change',
       'time_unix', 'ABER_ZG_5_15', 'ABER_SG_5_15',
       ...
       'HA_ema15', 'HA_pct_diff_ema5_15', 'HA_rsi', 'HA_macd', 'HA_macds',
       'HA_macdh', 'HA_cci', 'HA_atr', 'HA_ha_close_bbp50_std', 'HA_mfi'],
      dtype='object', length=311)


TypeError: Argument 'real' has incorrect type (expected numpy.ndarray, got tuple)

## Univariate Feature Selection Process

In [ ]:
# Re-scale the data to include the new feature
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# feature selection
selector = SelectKBest(score_func=f_classif, k="all")
selector.fit(X_scaled, y)

# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
features_df_new = X.iloc[:, cols]

# Store the scores of each feature in a dictionary
feature_scores = {
    feature_name: score for feature_name, score in zip(X.columns, selector.scores_)
}

# Sort the dictionary by value in descending order and print the scores
for feature_name, score in sorted(
    feature_scores.items(), key=lambda item: item[1], reverse=True
):
    print(f"{feature_name}: {score}")

# Now we can apply Logistic Regression and Random Forests on the new features_df_new
# Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=500)

# Cross-validation
cv_scores = cross_val_score(log_reg, features_df_new, y, cv=tscv, scoring="f1")

print(f"\nLogistic Regression CV F1 score: {np.mean(cv_scores)}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv_scores = cross_val_score(rf, features_df_new, y, cv=tscv, scoring="f1")

print(f"Random Forest CV F1 score: {np.mean(cv_scores)}")

# Reshape input to be 3D [samples, timesteps, features]
X_array = X.values
X_reshaped = X_array.reshape((X_array.shape[0], 1, X_array.shape[1]))

# Call the function
mean_f1_score = timeseries_cv_score(X_reshaped, y.values, n_splits=5)
print(f"\nLSTM CV F1 score: {mean_f1_score}")

print("\n", features_df_new.columns)

HA_body: 294.38880119737655
THERMOs_20_2_0.5: 273.60362108048645
THERMO_20_2_0.5: 270.249417080584
THERMOl_20_2_0.5: 157.06919674247334
volume_to_ma_volume: 95.47666501167771
volume: 91.47185887138413
HA_high_low: 90.77134390075592
PVOL: 90.50536446119777
turnover: 90.50536446119777
BBB_5_2.0: 79.13500197982869
std_5: 77.08391210633341
volatility_5: 77.08391210633341
ER_10: 76.49120671158205
vrsi: 66.81200791486042
volume_change: 54.94609293559068
PSARr_0.02_0.2: 42.35299874492656
TRUERANGE_1: 40.93139905386556
PVOh_10_50_9: 40.746005572468654
PDIST: 38.215663834041884
PVOh_12_26_9: 36.55651784395762
high_pct: 32.9043488025379
volume_ema5: 29.801691211064494
KURT_30: 27.621189680766168
PVO_10_50_9: 24.851073575058958
volume_std_5: 23.66638808822487
PVO_12_26_9: 23.58845909024736
PVR: 22.44330032229265
volume_change_roll5: 18.194421025682455
volume_ma_5: 16.86198277783865
volume_roll5: 16.86198277783865
std_10: 15.569841528761476
VHF_28: 14.526500737766861
DMN_14: 11.907639542415312
THE

c:\Users\jnorm\Projects\websocket_trading\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 46  50  52  55  57  58  77  80  81  82  87  90 101 104 105] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\jnorm\Projects\websocket_trading\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jnorm\Projects\websocket_trading\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jnorm\Projects\websocket_trading\.venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1204, in fit
    check_classification_targets(y)
  File "c:\Users\jnorm\Projects\websocket_trading\.venv\lib\site-packages\sklearn\utils\multiclass.py", line 218, in check_classification_targets
    raise ValueError("Unknown label type: %r" % y_type)
ValueError: Unknown label type: 'continuous'


## Base Model

In [ ]:
# Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=500)

# Cross-validation
cv_scores = cross_val_score(log_reg, X, y, cv=tscv, scoring="roc_auc")

print(f"Logistic Regression CV ROC AUC score: {np.mean(cv_scores)}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv_scores = cross_val_score(rf, X, y, cv=tscv, scoring="roc_auc")

print(f"Random Forest CV ROC AUC score: {np.mean(cv_scores)}")

print("\n", X.columns)

Logistic Regression CV ROC AUC score: 0.5516985893932947
Random Forest CV ROC AUC score: 0.9951631481367236

 Index(['open', 'close', 'high', 'low', 'volume', 'color', 'time',
       'ABER_ZG_5_15', 'ABER_SG_5_15', 'ABER_XG_5_15',
       ...
       'price_ema5', 'volume_ema5', 'price_to_ema5', 'volume_change_roll5',
       'avg_vol_last_100', 'turnover', 'hour', 'minute', 'day', 'month'],
      dtype='object', length=484)


## LSTM

In [ ]:
# Reshape input to be 3D [samples, timesteps, features]
X_array = X.values
X_reshaped = X_array.reshape((X_array.shape[0], 1, X_array.shape[1]))

# Call the function
mean_auc_score = timeseries_cv_score(X_reshaped, y.values, n_splits=5)
print(f"\nLSTM CV ROC AUC score: {mean_auc_score}")

42/42 [==============================] - 0s 720us/step

LSTM CV ROC AUC score: (0.6953945419128389, 0.4942367515727568)
